# IFRS17 シミュレーション モデル説明

## このNotebookについて

`ifrs17sim`モデルの説明のためのNotebookです。https://lifelib.io のプロジェクトを、ムーンライトセミナー用に作成したものです。


## モデルの概要

* `ifrs17sim`モデルは、IFRS17ベースのプロジェクションを、単一のモデルポイント上でシミュレーションするモデルです。
* リスク調整やTVOGはモデル化できていません。
* 個人の解釈で作成していますので、不正確な部分もあると思います。

## Spyder modelx プラグインの利用

* Spyder modelxプラグインを利用すると、モデルの階層構造を表示するウィジットが追加されます。
* Spyder modelxプラグインをインストールするには、Anaconda Promptを起動し、次のコマンドを入力します。

```
> pip install spyder_modelx
```
* View->Pane->Mx explorerを選択

![MxMenu](http://docs.modelx.io/en/latest/_images/SpyderMainMenuForModelx.png)

* Consoleのタブを右クリックし、"Open a modelx console"を選択

![MxConsole](http://docs.modelx.io/en/latest/_images/IPythonConsoleMenu.png)

* Mx ConsoleでのモデルがMx explorerウィジットに表示されます。

## モデルの作


* `ifrs17sim.py`モジュールの`build`関数で作成し、変数`model`に作成したモデルを代入します。変数名は任意です。
* ソースファイルから、数式をインポートし、インプットデータをエクセルファイル`input.xlsm`から読み込みます。
* ビルドすると、その状態がファイル`ifrs17sim.mx`というファイルに保存ます。次回から、`load_saved=True`と指定すれば、保存モデルを読み込むので時間が短縮できます。
* 複数回モデルを作成すると、以前のモデルは`ifrs17sim_BAKN`(Nは数字)という名前に変更されます。


In [ ]:
import ifrs17sim

model = ifrs17sim.build(load_saved=True)

## モデルの操作

変数名`model`に作成したモデル`ifrs17sim`が代入されています。

In [ ]:
model

全てのモデルの、名前とオブジェクトの対応を、`dict`で取得します。

In [ ]:
import modelx as mx

mx.get_models()

In [ ]:
mx.get_models()['ifrs17sim'] is model

## スペースの操作

モデルは複数のSpaceから構成されています。次の文は、`model`直下のスペースの名前とオブジェクトの対応を、`dict`で取得します。

In [ ]:
model.spaces

Spaceオブジェクトを取得します。

In [ ]:
model.OuterProj

Spaceの取得方法は、2通りあります。

In [ ]:
model.OuterProj is model.spaces['OuterProj']

スペースはさらにスペースを持つことができます。

In [ ]:
outer = model.OuterProj
outer.spaces

In [ ]:
outer.InnerProj.spaces

In [ ]:
outer.InnerProj.PresentValue

## ifrs17simモデルのスペースの内容

| Space      | 内容                                              | パラメータ |
|------------|---------------------------------------------------|-----------|
| Input      | input.xlsmから読み込まれた入力データ                |           |
| LifeTable  | 基数や脱退残存表。保険料率や解約返戻金率の計算に使用   |          |
| OuterProj  | 実績のプロジェクション                              |Poicy ID |
| Policy     | 契約属性                                           |Poicy ID |
| Assumption | 計算前提                                           |Poicy ID |
| Economic   | 金利シナリオ                                       |          |
| BaseProj   | プロジェクションのベーススペース                     |          |
| IFRS       | IFRS関連のベーススペース                            |          |
| OuterProj  | 実績プロジェクション                                |Poicy ID, Scen ID=1 |



## ベーススペース

他のスペースを**継承**することで、そのスペースのセルや子スペースを自分の中にコピーします。コピー元のスペースを、コピー先のスペースのベース(基底)スペースといいます。
例えば、`OuterProj`は、`BaseProj`と、`IFRS`を継承しています。ベーススペースは、`bases`属性で知ることができます。

In [ ]:
outer.bases

## パラメータ化されたスペースとダイナミックスペース

パラメータ化されたスペースは、鍵括弧で引数を指定すると、その引数に対応する子スペースが作成されます。

In [ ]:
outer.parameters

In [ ]:
outer[1]

## セルズの操作

`Cells`オブジェクトはSpaceオブジェクトに含まれています。

In [ ]:
outer.cells

In [ ]:
outer.ProfitBefTax

Space同様、Cellsオブジェクトには2通りの取得方法があります。

In [ ]:
outer.cells['ProfitBefTax'] is outer.ProfitBefTax

In [ ]:
model.Policy[171].AnnPremRate()

## レファレンス

スペース内のセルから別のスペースを参照するために、そのスペースから参照される他のスペースはrefで取得できます。

In [ ]:
outer[1].refs